# Signal profiles

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from math import fabs, ceil

# sns.set_style("whitegrid")
sns.set_style("white")
import os
from tqdm.auto import tqdm
from itertools import product
import os
import pyBigWig
from math import ceil
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

# GSE26320 reprocessed

In [ ]:
GSE26320_PATH = os.path.expanduser('~/data/2023_GSE26320')
GSE26320_MODIFICATIONS = ['CTCF',  'H3K4me3', 'H3K4me1', 'H3K27ac', 'H3K36me3', 'H3K27me3']
# GSE26320_CELLS = ['GM12878', 'HMEC', 'HSMM', 'K562', 'NHEK', 'NHLF', 'H1', 'Huvec', 'HepG2']
GSE26320_CELLS = ['GM12878', 'K562', 'NHEK']
# GSE26320_REPS = ['rep1', 'rep2']
GSE26320_REPS = ['rep1']

## Peaks length for ENCODE dataset

# Autocorrection across all coverage - with control correction
Paper: Retrieving Chromatin Patterns from Deep Sequencing Data Using Correlation Functions
https://www.cell.com/biophysj/fulltext/S0006-3495(17)30032-2#sec2

`bwext` folder is used to create bigwig visualization by enlarging coverage to fragment size.
```
# Visualization
for F in *.bam; do echo $F; samtools index $F; bamCoverage --ignoreDuplicates --extendReads 200 -b $F -p 6 -o ${F/.bam/_ext.bw}; done
```

In [ ]:
CHROM_SIZES = {
    c: s for _, (c, s) in pd.read_csv(os.path.join(GSE26320_PATH, 'hg38.chrom.sizes'),
                                      sep='\t', names=['chr', 'size']).iterrows()
}
CHROM_SIZES

In [ ]:
GSE26320_BW_PATH = os.path.join(GSE26320_PATH, 'bwext')

def r0(o, c):
    """ Correlation to estimate b """
    assert o.size == c.size
    # o_mean = o.sum() / o.size
    # c_mean = c.sum() / c.size
    # on = o - o_mean
    # cn = c - c_mean
    # return np.dot(on, cn) / sqrt(np.dot(on, on) * np.dot(cn, cn))
    return pearsonr(o, c)[0]

def rDx(dx, o1, o2):
    """ Slow shifted correlation function by formula from paper """
    assert o1.size == o2.size
    # n = o1.size
    # o1_mean = o1.sum() / o1.size
    # o2_mean = o2.sum() / o2.size
    # o1n = o1 - o1_mean
    # o2n = o2 - o2_mean
    # nominator = (1 / (2 * (n - dx))) * sum(
    #     (o1[i] - o1_mean) * (o2[i + dx] - o2_mean) + (o1[i + dx] - o1_mean) * (o2[i] - o2_mean)
    #     for i in range(n - dx)
    # )
    # denominator = (1 / n) * sqrt(np.dot(o1n, o1n) * np.dot(o2n, o2n))
    # return nominator / denominator
    return pearsonr(o1, np.roll(o2, dx))[0]

# TODO: support blacklisted regions to avoid long-distance zero coverage

def compute_autocorrelations_paper(modifications, cells, replicates, chrom_sizes, bin, ds, control='Input', bwpath=GSE26320_BW_PATH):
    correlations = []
    for mod, cell, rep in product(modifications, cells, replicates):
        bwfile = next((f for f in os.listdir(bwpath) if '.bw' in f and mod in f and cell in f and rep in f), None)
        bwcfile = next((f for f in os.listdir(bwpath) if '.bw' in f and control in f and cell in f and rep in f), None)
        if bwfile is None or bwcfile is None:
            continue
        print(mod, cell, rep, bwfile, bwcfile)
        for chr, size in tqdm(chrom_sizes.items()):
            with pyBigWig.open(os.path.join(bwpath, bwfile)) as bw:
                coverage = np.asarray(bw.stats(chr, nBins=(int(ceil(size / bin))), exact=True, type='sum'))
            with pyBigWig.open(os.path.join(bwpath, bwcfile)) as bwc:
                coveragec = np.asarray(bwc.stats(chr, nBins=(int(ceil(size / bin))), exact=True, type='sum'))
            size = min(size, 50_000_000)
            print(f'Computing {mod} {cell} {rep} binned {bin}bp coverage correlations on chr {chr} {size}')
            # The coverage was initially calculated for each chromosome by extending the reads to
            # fragment length, yielding a histogram with the genomic coordinate on the x axis
            # and the number of reads per basepair on the y axis.

            print(f'{chr} Coverage mln signal: {coverage.sum()} control: {coveragec.sum()}')
            # First, the normalized coverage of the control Cnorm and of the specific IP Anorm
            # were obtained by dividing by input signal I. Positions with zero input coverage were neglected.
            # Subsequently, the coverage at these positions was set to the respective average value
            # that was calculated for the remaining positions, thus eliminating fluctuations and
            # corresponding contributions to the correlation coefficient from these positions.
            coverage_mean = coverage.sum() / sum(coverage > 0)
            coveragec_mean = coveragec.sum() / sum(coveragec > 0)
            print(f'Mean signal {coverage_mean} control {coveragec_mean}')
            coverage[coverage == 0] = coverage_mean
            coveragec[coveragec == 0] = coveragec_mean
            coverage = coverage / coverage_mean
            coveragec = coveragec / coveragec_mean
            # In the next step, nonspecific background signal was removed to obtain the
            # normalized read occupancy O = Anorm − b × Cnorm
            # The parameter b quantifies the contribution of the control signal present as
            # background in the sample (IP).
            # To estimate b, we minimized the absolute value of the Pearson correlation coefficient r0 at
            # zero shift distance between the normalized occupancy O and the control coverage Cnorm.
            # For the minimization procedure, b was changed between 0 and 1.
            print('Estimating b between 0 and 1')
            stepB = 0.001
            bs = np.linspace(0, 1, num = int(1 / stepB))
            bCs = [fabs(r0(coverage - b * coveragec, coveragec)) for b in bs]
            bI = np.argmin(bCs)
            b = bs[bI]
            print(f'I={bI}, b={b}, correlation={bCs[bI]}')
            coverageo = coverage - b * coveragec
            print('Computing autocorrelations')
            for d in ds:
                corr, pval = rDx(d, coverageo, coverageo), None
                correlations.append((mod, cell, rep, bwfile, chr, coverage_mean, coveragec_mean, b,
                                     d * bin, corr, pval))
    return pd.DataFrame(
        columns=['modification', 'cell', 'replicate', 'file', 'chr', 'coverage mean', 'control mean', 'b',
                 'd', 'correlation', 'pvalue'],
        data=correlations
    )

In [ ]:
# To sample the correlation function in a quasi-logarithmic manner (33),
# profiles were binned by a factor of two after 25 shift operations to double the step size.
# To preserve high resolution for small shift distances, the first binning operation was carried
# out at a shift of Δx = 50 bp.
BIN = 50
DS = list(range(0, int(5000 / BIN))) + \
     list(range(int(5000 / BIN), int(100000 / BIN), int(100 / BIN))) + \
     list(range(int(10000 / BIN), int(1000000 / BIN), int(10000 / BIN)))
print(len(DS))

In [ ]:
chr_sizes = {'chr22': CHROM_SIZES['chr22']}
df_correlations = compute_autocorrelations_paper(GSE26320_MODIFICATIONS, GSE26320_CELLS, GSE26320_REPS,
                                                 {'chr22': CHROM_SIZES['chr22']}, BIN, DS)
df_correlations

In [ ]:
plt.figure(figsize=(5, 3))
ax = plt.axes()
# Show aggregated data, since we don't want error plots
t = df_correlations.copy()
t.loc[t['d'] == 0, 'd'] = BIN / 2
t = t[['modification', 'd', 'correlation']].groupby(['modification', 'd']).mean().reset_index()
g_results = sns.lineplot(data=t, x="d", y="correlation", hue="modification",
                         hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3'])
# sample_count = [1, 10, 50, 100, 200, 500, 1000, 2000, 5000,
#                 10_000, 20_000, 50_000, 100_000, 200_000, 500_000, 1_000_000]
sample_count = [100, 1000, 10_000, 100_000, 1_000_000]

g_results.set(xscale='log')
g_results.set(xticks=sample_count)
g_results.set(xticklabels=sample_count)
ax.title.set_text(f'Auto correlation bin={BIN}')
ax.set_xlim(BIN / 2, 1_000_000)
plt.tight_layout()
plt.savefig(f'{GSE26320_PATH}/pics/paper_signal_correlation.pdf', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(5, 3))
ax = plt.axes()
sns.lineplot(data=t, x="d", y="correlation", hue="modification",
             hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3'])
ax.title.set_text(f'Auto correlation bin={BIN}')
ax.set_xlim(0, 5000)
ax.legend(loc='upper right')
plt.tight_layout()
plt.savefig(f'{GSE26320_PATH}/pics/paper_signal_correlation_5000.pdf', bbox_inches='tight', dpi=300)
plt.show()

# Signal profile along peaks

In [ ]:
RANGE = 50_000
BIN = 50
BINS = int(RANGE / BIN)
PEAKS = 1000

In [ ]:
GSE26320_SPAN_PATH = os.path.join(GSE26320_PATH, 'span')
GSE26320_BW_PATH = os.path.join(GSE26320_PATH, 'bwext')

RANGE = 10000
BIN = 50
BINS = int(RANGE / BIN)
PEAKS = 100

def intersect(c1, s1, e1, c2, s2, e2):
    return c1 == c2 and (s1 <= s2 < e1 or s1 <= e2 - 1 < e2)

def signal_peaks_profile(modifications, cells, replicates, peakspath=GSE26320_SPAN_PATH, bwpath=GSE26320_BW_PATH,
                         control='Input'):
    profiles = []
    for mod, cell, rep in tqdm(list(product(modifications, cells, replicates))):
        peaksfile = next((f for f in os.listdir(peakspath) if mod in f and cell in f and rep in f), None)
        bwfile = next((f for f in os.listdir(bwpath) if '.bw' in f and mod in f and cell in f and rep in f), None)
        bwcfile = next((f for f in os.listdir(bwpath) if '.bw' in f and control in f and cell in f and rep in f), None)
        if peaksfile is None or bwfile is None:
            continue
        print(mod, cell, rep, peaksfile, bwfile)
        # Load peaks file and sort by score
        peaksfile_df = pd.read_csv(os.path.join(peakspath, peaksfile), sep='\t',
                                   names=['chr', 'start', 'end', '4', '5', '6', '7', '8', 'score'])
        peaksfile_df = peaksfile_df[['chr', 'start', 'end', 'score']].copy()
        peaksfile_df.sort_values(by=['score'], ascending=False, inplace=True)
        # Ensure that peaks do not intersect and doesn't contain any significant peaks
        intersects = [t for _, t in peaksfile_df.head(PEAKS)[['chr', 'start', 'end']].iterrows()]
        added = 0
        with pyBigWig.open(os.path.join(bwpath, bwfile)) as bw:
            with pyBigWig.open(os.path.join(bwpath, bwcfile)) as bwc:
                total_coverage = sum(bw.stats(f'chr{i}', exact=True)[0] for i in range(1, 22))
                total_coveragec = sum(bwc.stats(f'chr{i}', exact=True)[0] for i in range(1, 22))
                # print(total_coverage)
                for _, (chr, start, end, score) in peaksfile_df.iterrows():
                    # No more than single intersection
                    if sum(intersect(chr, start, end, c, s, e) for c, s, e in intersects) > 1:
                        continue
                    # Find summit for narrow peak, center of peak for broad
                    if mod in ['CTCF', 'H3K27ac', 'H3K4me3']:
                        nBins = min(BINS, int((end - start) / 20))
                        stats = bw.stats(chr, start, end, nBins=nBins, exact=True)
                        max_offset = start + (np.argmax(stats) + 0.5) * (end - start) / nBins
                        r_start = int(max_offset  - RANGE / 2)
                    else:
                        r_start = int((start + end) / 2 - RANGE / 2)
                    if r_start < 0:
                        continue
                    r_end = r_start + RANGE
                    # Check that no peak intersect others
                    if sum(intersect(chr, r_start, r_end, c, s, e) for c, s, e in intersects) > 1:
                        continue
                    intersects.append((chr, r_start, r_end))
                    # Analyse signal and control coverage
                    try:
                        for i, cov in enumerate(bw.stats(chr, r_start, r_end, nBins=BINS, exact=True)):
                            profiles.append((mod, cell, rep, added, (-BINS / 2 + i) * BIN, cov / total_coverage))
                        for i, cov in enumerate(bwc.stats(chr, r_start, r_end, nBins=BINS, exact=True)):
                            profiles.append((control, cell, rep, added, (-BINS / 2 + i) * BIN, cov / total_coveragec))
                    except:
                        pass
                    added += 1
                    if added == PEAKS:
                        break

    return pd.DataFrame(columns=['modification', 'cell', 'replicate', 'peak', 'x', 'coverage'], data=profiles)

In [ ]:
df_gse26320_cov = signal_peaks_profile(GSE26320_MODIFICATIONS, GSE26320_CELLS, GSE26320_REPS)
df_gse26320_cov

In [ ]:
plt.figure(figsize=(5, 3))
# Show aggregated data, since we don't want error plots
t = df_gse26320_cov[['modification', 'x', 'coverage']].groupby(['modification', 'x']).mean().reset_index()
sns.lineplot(data=t, x="x", y="coverage", hue="modification", err_style=None,
             hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3'])
plt.tight_layout()
plt.savefig(f'{GSE26320_PATH}/pics/peaks_profile.pdf', bbox_inches='tight', dpi=300)
plt.show()

# Peaks densities

In [ ]:
GSE26320_SPAN_PATH = os.path.join(GSE26320_PATH, 'span')
GSE26320_BW_PATH = os.path.join(GSE26320_PATH, 'bwext')

data = []
for mod, cell, rep in tqdm(list(product(GSE26320_MODIFICATIONS, GSE26320_CELLS, GSE26320_REPS))):
    peaksfile = next((f for f in os.listdir(GSE26320_SPAN_PATH) if mod in f and cell in f and rep in f), None)
    bwfile = next((f for f in os.listdir(GSE26320_BW_PATH) if '.bw' in f and mod in f and cell in f and rep in f), None)
    bwcfile = next((f for f in os.listdir(GSE26320_BW_PATH) if '.bw' in f and 'Input' in f and cell in f and rep in f),
                   None)
    if peaksfile is None or bwfile is None or bwcfile is None:
        continue
    print(mod, cell, rep, peaksfile, bwfile, bwcfile)

    # Load peaks file and sort by score
    peaksfile_df = pd.read_csv(os.path.join(GSE26320_SPAN_PATH, peaksfile), sep='\t',
                               names=['chr', 'start', 'end', '4', '5', '6', '7', '8', 'score'])
    with pyBigWig.open(os.path.join(GSE26320_BW_PATH, bwfile)) as bw:
        with pyBigWig.open(os.path.join(GSE26320_BW_PATH, bwcfile)) as bwc:
            # Resort top scored peaks by density and start with the most dense ones
            total_coverage = sum(bw.stats(chr, exact=True, type='sum')[0] for chr in CHROM_SIZES.keys()) / 1e6
            total_coveragec = sum(bwc.stats(chr, exact=True, type='sum')[0] for chr in CHROM_SIZES.keys()) / 1e6
            for _, (chr, start, end) in peaksfile_df[['chr', 'start', 'end']].iterrows():
                peak_coverage = bw.stats(chr, start, end, exact=True, type='sum')[0]
                peak_coveragec = bwc.stats(chr, start, end, exact=True, type='sum')[0]
                data.append((mod, cell, rep, total_coverage, total_coveragec,
                             f'{chr}:{start}-{end}', end - start, peak_coverage, peak_coveragec))

gse26320_peak_coverages_df = pd.DataFrame(
    columns=['modification', 'cell', 'replicate', 'total_signal_coverage', 'total_control_coverage',
             'peak', 'length', 'signal_coverage', 'control_coverage'],
    data=data
)
del data
gse26320_peak_coverages_df

In [ ]:
plt.figure(figsize=(6, 4))
t = pd.concat([gse26320_peak_coverages_df[gse26320_peak_coverages_df['modification'] == m].sample(2_000)
               for m in gse26320_peak_coverages_df['modification'].unique()]).reset_index(drop=True).sample(frac=1).reset_index(drop=True)
t['RPKM'] = t['signal_coverage'] / t['length'] * 1e3
t['RPKM'] /= t['total_signal_coverage']
t['RPKM'].clip(lower=0.2, inplace=True)
t['length'].clip(upper=3e5, inplace=True)
sample_count = [100,  1000, 10_000, 100_000]
g_results = sns.scatterplot(data=t, x='length', y='RPKM', hue='modification', alpha=0.3,
                            hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3'])
# g_results = sns.kdeplot(data=t, x='length', y='RPKM', hue='modification', alpha=0.3,
#                         hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3'], fill='true')

g_results.set(xscale='log')
g_results.set(xticks=sample_count)
g_results.set(xticklabels=sample_count)
g_results.set(yscale='log')
# Put a legend to the right of the current axis
g_results.axes.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.savefig(f'{GSE26320_PATH}/pics/peaks_length_vs_rpkm.pdf', bbox_inches='tight', dpi=300)
plt.show()

# Autocorrelations - without control correction
Simplified autocorrelation computation

In [ ]:
def compute_autocorrelations(modifications, cells, replicates, chrom_sizes, bin, max_d, bwpath=GSE26320_BW_PATH):
    correlations = []
    for mod, cell, rep in tqdm(list(product(modifications, cells, replicates))):
        bwfile = next((f for f in os.listdir(bwpath) if '.bw' in f and mod in f and cell in f and rep in f), None)
        if bwfile is None:
            continue
        print(mod, cell, rep, bwfile, bwcfile)
        print(f'Computing binned coverage correlations')
        with pyBigWig.open(os.path.join(bwpath, bwfile)) as bw:
            total_coverage = sum(bw.stats(chr, exact=True, type='sum')[0] for chr in CHROM_SIZES.keys()) / 1e6
            print(f'Signal coverage mln {total_coverage} control {total_coveragec}')
            for chr, size in tqdm(chrom_sizes.items()):
                # TODO: support blacklisted regions to avoid long-distance zero coverage
                coverage = np.asarray(bw.stats(chr, 0, size, nBins=(int(ceil((size) / bin))),
                                               exact=True, type='sum'))
                # Ignore non-covered regions
                coverage = coverage[coverage > 0]
                for d in range(0, max_d):
                    corr, pval = pearsonr(coverage, np.roll(coverage, d))
                    correlations.append((mod, cell, rep, bwfile, chr, d * bin, corr, pval))
    return pd.DataFrame(
        columns=['modification', 'cell', 'replicate', 'file', 'chr', 'd', 'correlation', 'pvalue'],
        data=correlations
    )

In [ ]:
df_correlations = compute_autocorrelations(['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3', 'Input'],
                                           GSE26320_CELLS, ['rep1'], {'chr22': CHROM_SIZES['chr22']}, BIN, 5000)
df_correlations

In [ ]:
plt.figure(figsize=(5, 3))
ax = plt.axes()
# Show aggregated data, since we don't want error plots
t = df_correlations.copy()
t.loc[t['d'] == 0, 'd'] = BIN - 1
t = df_correlations[['modification', 'd', 'correlation']].groupby(['modification', 'd']).mean().reset_index()
g_results = sns.lineplot(data=t, x="d", y="correlation", hue="modification",
                         hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3', 'Input'])
sample_count = [100,  1000, 10_000, 100_000]
g_results.set(xscale='log')
g_results.set(xticks=sample_count)
g_results.set(xticklabels=sample_count)
ax.title.set_text(f'Auto correlation bin={BIN}')
ax.set_xlim(BIN - 1, 100_000)
ax.legend(loc='upper right')
plt.tight_layout()
plt.savefig(f'{GSE26320_PATH}/pics/signal_correlation.pdf', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(5, 3))
ax = plt.axes()
sns.lineplot(data=t, x="d", y="correlation", hue="modification",
             hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3', 'Input'])
ax.title.set_text(f'Auto correlation bin={BIN}')
ax.set_xlim(0, 5000)
plt.tight_layout()
plt.savefig(f'{GSE26320_PATH}/pics/signal_correlation_5000.pdf', bbox_inches='tight', dpi=300)
plt.show()

# RoadmapEpigenomics autocorrelation

In [ ]:
PATH = os.path.expanduser('~/data/2023_Immune')

IMMUNE_CELLS = ['BCell', 'TCell', 'Monocyte']
MODIFICATIONS = ['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3']

In [ ]:
chr_sizes = {'chr22': CHROM_SIZES['chr22']}
df_correlations = compute_autocorrelations_paper(MODIFICATIONS, IMMUNE_CELLS, [''],
                                                 {'chr22': CHROM_SIZES['chr22']},
                                                 BIN, DS, control='Control', bwpath=PATH + '/bwext')
df_correlations

In [ ]:
plt.figure(figsize=(5, 3))
ax = plt.axes()
# Show aggregated data, since we don't want error plots
t = df_correlations.copy()
t.loc[t['d'] == 0, 'd'] = BIN / 2
t = t[['modification', 'd', 'correlation']].groupby(['modification', 'd']).mean().reset_index()
g_results = sns.lineplot(data=t, x="d", y="correlation", hue="modification",
                         hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3'])
# sample_count = [1, 10, 50, 100, 200, 500, 1000, 2000, 5000,
#                 10_000, 20_000, 50_000, 100_000, 200_000, 500_000, 1_000_000]
sample_count = [100, 1000, 10_000, 100_000, 1_000_000]

g_results.set(xscale='log')
g_results.set(xticks=sample_count)
g_results.set(xticklabels=sample_count)
ax.title.set_text(f'Auto correlation bin={BIN}')
ax.set_xlim(BIN / 2, 1_000_000)
plt.tight_layout()
plt.savefig(f'{PATH}/pics/paper_signal_correlation.pdf', bbox_inches='tight', dpi=300)
plt.show()

## Peaks profile

In [ ]:
IMMUNE_SPAN_PATH = os.path.join(PATH, 'span')
IMMUNE_BW_PATH = os.path.join(PATH, 'bwext')

df_immune_cov = signal_peaks_profile(MODIFICATIONS, IMMUNE_CELLS, [''], IMMUNE_SPAN_PATH, IMMUNE_BW_PATH, 'Control')
df_immune_cov

In [ ]:
plt.figure(figsize=(5, 3))
# Show aggregated data, since we don't want error plots
t = df_gse26320_cov[['modification', 'x', 'coverage']].groupby(['modification', 'x']).mean().reset_index()
sns.lineplot(data=t, x="x", y="coverage", hue="modification", err_style=None,
             hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3'])
plt.tight_layout()
plt.savefig(f'{PATH}/pics/peaks_profile.pdf', bbox_inches='tight', dpi=300)
plt.show()

## Peaks intensities

In [ ]:
data = []
for mod, cell, rep in tqdm(list(product(MODIFICATIONS, IMMUNE_CELLS, ['']))):
    peaksfile = next((f for f in os.listdir(IMMUNE_SPAN_PATH) if mod in f and cell in f and rep in f), None)
    bwfile = next((f for f in os.listdir(IMMUNE_BW_PATH) if '.bw' in f and mod in f and cell in f and rep in f), None)
    bwcfile = next((f for f in os.listdir(IMMUNE_BW_PATH) if '.bw' in f and 'Control' in f and cell in f and rep in f), None)
    if peaksfile is None or bwfile is None or bwcfile is None:
        continue
    print(mod, cell, rep, peaksfile, bwfile, bwcfile)

    # Load peaks file and sort by score
    peaksfile_df = pd.read_csv(os.path.join(IMMUNE_SPAN_PATH, peaksfile), sep='\t',
                               names=['chr', 'start', 'end', '4', '5', '6', '7', '8', 'score'])
    with pyBigWig.open(os.path.join(IMMUNE_BW_PATH, bwfile)) as bw:
        with pyBigWig.open(os.path.join(IMMUNE_BW_PATH, bwcfile)) as bwc:
            # Resort top scored peaks by density and start with the most dense ones
            total_coverage = sum(bw.stats(chr, exact=True, type='sum')[0] for chr in CHROM_SIZES.keys() if '_' not in chr) / 1e6
            total_coveragec = sum(bwc.stats(chr, exact=True, type='sum')[0] for chr in CHROM_SIZES.keys() if '_' not in chr) / 1e6
            for _, (chr, start, end) in peaksfile_df[['chr', 'start', 'end']].iterrows():
                if '_' in chr:
                    continue
                peak_coverage = bw.stats(chr, start, end, exact=True, type='sum')[0]
                peak_coveragec = bwc.stats(chr, start, end, exact=True, type='sum')[0]
                data.append((mod, cell, rep, total_coverage, total_coveragec,
                             f'{chr}:{start}-{end}', end - start, peak_coverage, peak_coveragec))

immune_peak_coverages_df = pd.DataFrame(
    columns=['modification', 'cell', 'replicate', 'total_signal_coverage', 'total_control_coverage',
             'peak', 'length', 'signal_coverage', 'control_coverage'],
    data=data
)
del data
immune_peak_coverages_df

In [ ]:
plt.figure(figsize=(6, 4))
t = pd.concat([immune_peak_coverages_df[immune_peak_coverages_df['modification'] == m].sample(2_000)
               for m in immune_peak_coverages_df['modification'].unique()]).reset_index(drop=True).sample(frac=1).reset_index(drop=True)
t['RPKM'] = t['signal_coverage'] / t['length'] * 1e3
t['RPKM'] /= t['total_signal_coverage']
t['RPKM'].clip(lower=0.2, inplace=True)
t['length'].clip(upper=3e5, inplace=True)
sample_count = [100,  1000, 10_000, 100_000]
g_results = sns.scatterplot(data=t, x='length', y='RPKM', hue='modification', alpha=0.3,
                            hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3'])
# g_results = sns.kdeplot(data=t, x='length', y='RPKM', hue='modification', alpha=0.3,
#                             hue_order=['H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K27me3'], fill='true')
g_results.set(xscale='log')
g_results.set(xticks=sample_count)
g_results.set(xticklabels=sample_count)
g_results.set(yscale='log')
# Put a legend to the right of the current axis
g_results.axes.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.savefig(f'{PATH}/pics/peaks_length_vs_rpkm.pdf', bbox_inches='tight', dpi=300)
plt.show()